In [1]:
import pandas as pd
from pathlib import Path



csv_path = Path(r"C:\Users\ahern\Desktop\Warehouse Project\data\garments_worker_productivity.csv")

df = pd.read_csv(csv_path)

print("Shape:", df.shape)
df.head()

Shape: (1197, 15)


,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [2]:

df = df.copy()
# Convert date column to datetime, coerce errors to NaT
df["date"] = pd.to_datetime(df["date"], errors="coerce")

#  fixes typo 'sweing' to 'sewing' in department column from orginal dataset
df["department"] = (
    df["department"]
    .astype(str).str.strip()
    .str.replace(r"^\s*sweing\s*$", "sewing", regex=True)
)

# Ordered weekdays which helps plotting & Power BI sorting later
dow = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
df["day"] = pd.Categorical(df["day"], categories=dow, ordered=True)



In [3]:
# Impute missing wip with median, i dont think we'll use WIP but I want to show we can fix missing values before outoput
if "wip" in df.columns:
    wip_median = df["wip"].median(skipna=True)
    df["wip"] = df["wip"].fillna(wip_median)


In [4]:
# Clip ranges of productivity columns to sensible values,
# e.g. productivity should be between 0 and 1 (0% to 100%)
df["actual_productivity"] = df["actual_productivity"].clip(lower=0, upper=1)
df["targeted_productivity"] = df["targeted_productivity"].clip(lower=0, upper=1)

# Ensures numeric columns are numeric 
numeric_cols = [
    "team","targeted_productivity","smv","wip","over_time","incentive",
    "idle_time","idle_men","no_of_style_change","no_of_workers","actual_productivity"
]
for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

In [5]:
out_clean = Path(r"C:\Users\ahern\Desktop\Warehouse Project\exports\garment_productivity_clean_latest.csv")
out_clean.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_clean, index=False)
print("Saved", out_clean)

Saved C:\Users\ahern\Desktop\Warehouse Project\exports\garment_productivity_clean_latest.csv


In [6]:
import numpy as np

kpi = df.groupby(['date','department','team'], as_index=False).agg(
    actual_productivity=('actual_productivity','mean'),
    targeted_productivity=('targeted_productivity','mean'),
    smv=('smv','mean'),
    wip=('wip','mean'),
    over_time=('over_time','sum'),
    incentive=('incentive','mean'),
    idle_time=('idle_time','mean'),
    idle_men=('idle_men','mean'),
    no_of_style_change=('no_of_style_change','mean'),
    no_of_workers=('no_of_workers','mean'),
)

# Safe division avoids inf when target is 0 this is to ensure no errors in Power BI
kpi['perf_ratio'] = np.where(
    kpi['targeted_productivity'] > 0,
    kpi['actual_productivity'] / kpi['targeted_productivity'],
    np.nan
)

# Round for nicer BI display (4 decimal places = 0.01% precision)
kpi['perf_ratio'] = kpi['perf_ratio'].round(4)

# Save
out_kpi = r"C:\Users\ahern\Desktop\Warehouse Project\exports\kpi_latest.csv"
kpi.to_csv(out_kpi, index=False)
print("Saved", out_kpi,)

Saved C:\Users\ahern\Desktop\Warehouse Project\exports\kpi_latest.csv


In [7]:
# Random Forest Regression to predict actual productivity

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

#  drop 'actual_productivity' for the predictors and convert categoricals
X = pd.get_dummies(
    df.drop(columns=['actual_productivity','date']),  # date dropped for simplicity
    drop_first=True
)
y = df['actual_productivity']

# Split and train
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = RandomForestRegressor(n_estimators=150, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Evaluate on the test set
mae = mean_absolute_error(y_test, model.predict(X_test))
print("Prediction MAE (mean absolute error):", round(mae, 4))

# Make predictions 
df_pred = df.copy()
df_pred['predicted_productivity'] = model.predict(X)
df_pred['model_mae'] = round(mae, 4)  # include MAE as a reference column w

# Save predictions 
from pathlib import Path
pred_out = Path(r"C:\Users\ahern\Desktop\Warehouse Project\exports\predictions_latest.csv")
pred_out.parent.mkdir(parents=True, exist_ok=True)
df_pred.to_csv(pred_out, index=False)
print("Saved", pred_out)

Prediction MAE (mean absolute error): 0.0726
Saved C:\Users\ahern\Desktop\Warehouse Project\exports\predictions_latest.csv


In [8]:
#Here is where we combine the kpi and predictions for easier use in Power BI later

from pathlib import Path

# Load the two latest CSVs if the dataframs are not already in memory, just in case
kpi = pd.read_csv(r"C:\Users\ahern\Desktop\Warehouse Project\exports\kpi_latest.csv")
pred = pd.read_csv(r"C:\Users\ahern\Desktop\Warehouse Project\exports\predictions_latest.csv")

combined = pd.merge(
    kpi,
    pred[['date','department','team','predicted_productivity','model_mae']],
    on=['date','department','team'],
    how='left'
)

out_path = Path(r"C:\Users\ahern\Desktop\Warehouse Project\exports\kpi_with_predictions_latest.csv")
combined.to_csv(out_path, index=False)
print("Combined and Saved ", out_path)

Combined and Saved  C:\Users\ahern\Desktop\Warehouse Project\exports\kpi_with_predictions_latest.csv


In [9]:
# Feature importance from the Random Forest model, this was an after thought
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Get feature importances from model
importances = model.feature_importances_
features = X.columns

feat_imp = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)


# Save 
feat_out = Path(r"C:\Users\ahern\Desktop\Warehouse Project\exports\feature_importance.csv")
feat_imp.to_csv(feat_out, index=False)
print("Feature importances saved", feat_out)


#Test text for "output.ipynb" 111222333



Feature importances saved C:\Users\ahern\Desktop\Warehouse Project\exports\feature_importance.csv
